# Extract. Transform. Analysis.

In [74]:
import pandas as pd
#import numpy as np
import requests
import country_converter as coco
from bs4 import BeautifulSoup
import folium

### Extract

За основу беру текущий индекс Numbeo "Current Cost of Living Index", в котором на данный момент 506 городов и, пока, не понятно сколько стран.

In [75]:
url = 'https://www.numbeo.com/cost-of-living/rankings_current.jsp'

In [76]:
df = pd.read_html(url)[1]

In [77]:
df.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42
1,NaN,"Zurich, Switzerland",120.28,61.53,92.49,122.67,114.61,115.32
2,NaN,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13
3,NaN,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38
4,NaN,"Bern, Switzerland",112.70,36.46,76.63,114.77,105.62,116.10


Добавляю в таблицу ссылки на страницы данных городов

In [78]:
response = requests.get(url).text
soup = BeautifulSoup(response, 'lxml')
links = soup.find_all('td', class_='cityOrCountryInIndicesTable')

href_list = []
for i in links:
    href_list.append([i.find('a').text, i.find('a').get('href')])
    
city_link = pd.DataFrame(href_list, columns=['City', 'link'])

df = df.merge(city_link, on='City', how='left')

### Transform

In [79]:
df.drop('Rank', axis=1, inplace=True)

In [80]:
df.head()

,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link
0,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,https://www.numbeo.com/cost-of-living/in/Hamil...
1,"Zurich, Switzerland",120.28,61.53,92.49,122.67,114.61,115.32,https://www.numbeo.com/cost-of-living/in/Zurich
2,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,https://www.numbeo.com/cost-of-living/in/Basel
3,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38,https://www.numbeo.com/cost-of-living/in/Zug
4,"Bern, Switzerland",112.70,36.46,76.63,114.77,105.62,116.10,https://www.numbeo.com/cost-of-living/in/Bern


#### Разделить Город - Штат - Страна

In [81]:
split_table1 = df['City'].str.rsplit(', ', n=1, expand=True)
split_table1.columns=['city_state', 'country']

split_table2 = split_table1['city_state'].str.split(', ', n=1, expand=True)
split_table2.columns=['city', 'state']

In [82]:
split_table2

,city,state
0,Hamilton,None
1,Zurich,None
2,Basel,None
3,Zug,None
4,Bern,None
...,...,...
502,Islamabad,None
503,Lahore,None
504,Karachi,None
505,Rawalpindi,None


In [83]:
df = pd.concat([split_table2, split_table1['country'], df],axis=1)

In [84]:
df.head()

,city,state,country,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,https://www.numbeo.com/cost-of-living/in/Hamil...
1,Zurich,None,Switzerland,"Zurich, Switzerland",120.28,61.53,92.49,122.67,114.61,115.32,https://www.numbeo.com/cost-of-living/in/Zurich
2,Basel,None,Switzerland,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,https://www.numbeo.com/cost-of-living/in/Basel
3,Zug,None,Switzerland,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38,https://www.numbeo.com/cost-of-living/in/Zug
4,Bern,None,Switzerland,"Bern, Switzerland",112.70,36.46,76.63,114.77,105.62,116.10,https://www.numbeo.com/cost-of-living/in/Bern


In [85]:
df['state'].nunique()

36

In [86]:
df['state'].unique()

array([None, 'HI', 'CA', 'NY', 'AK', 'WA', 'DC', 'MA', 'CO', 'OR', 'FL',
       'TN', 'BC', 'IL', 'NC', 'MN', 'TX', 'LA', 'PA',
       'Newfoundland and Labrador', 'NE', 'GA', 'MD', 'AZ', 'NV', 'OH',
       'IN', 'MI', 'WI', 'KY', 'ID', 'MO', 'UT', 'OK', 'AR', 'KS',
       'Ajara'], dtype=object)

In [87]:
df[df['state'].notnull() & (df['country'] != "United States")]

,city,state,country,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link
53,Nanaimo,BC,Canada,"Nanaimo, BC, Canada",77.47,37.20,58.42,81.15,66.41,90.51,https://www.numbeo.com/cost-of-living/in/Nanaimo
72,St. John's,Newfoundland and Labrador,Canada,"St. John's, Newfoundland and Labrador, Canada",75.16,22.95,50.46,74.43,71.00,141.04,https://www.numbeo.com/cost-of-living/in/St-Jo...
450,Batumi,Ajara,Georgia,"Batumi, Ajara, Georgia",32.25,12.79,23.04,31.50,27.24,26.18,https://www.numbeo.com/cost-of-living/in/Batumi


#### Добавляю alpha_3 код к странам
Добавляю через lambda к каждой строке. Не самый эффективный вариант, обрабатывается капждая строка, страны дублируются, но просто попробовать (merge уже был).

In [88]:
df['country_code'] = df.apply(lambda row: coco.convert(names=row.country, to='ISO3') , axis = 1)

In [89]:
df.head(3)

,city,state,country,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link,country_code
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,https://www.numbeo.com/cost-of-living/in/Hamil...,BMU
1,Zurich,None,Switzerland,"Zurich, Switzerland",120.28,61.53,92.49,122.67,114.61,115.32,https://www.numbeo.com/cost-of-living/in/Zurich,CHE
2,Basel,None,Switzerland,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,https://www.numbeo.com/cost-of-living/in/Basel,CHE


#### Добавляю код к штатам USA

Сработало! Привел штаты Америки к стандарту

In [90]:
indexes = df[df['state'].notnull() & (df['country'] != "United States")]['state'].index
df.loc[indexes, 'state'] = None

In [91]:
df['state'] = df.apply(lambda row: f"US-{row['state']}" if row['state'] != None else None, axis = 1)

In [92]:
df.head(10)

,city,state,country,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index,link,country_code
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,https://www.numbeo.com/cost-of-living/in/Hamil...,BMU
1,Zurich,None,Switzerland,"Zurich, Switzerland",120.28,61.53,92.49,122.67,114.61,115.32,https://www.numbeo.com/cost-of-living/in/Zurich,CHE
2,Basel,None,Switzerland,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,https://www.numbeo.com/cost-of-living/in/Basel,CHE
3,Zug,None,Switzerland,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38,https://www.numbeo.com/cost-of-living/in/Zug,CHE
4,Bern,None,Switzerland,"Bern, Switzerland",112.70,36.46,76.63,114.77,105.62,116.10,https://www.numbeo.com/cost-of-living/in/Bern,CHE
5,Lausanne,None,Switzerland,"Lausanne, Switzerland",111.67,53.34,84.07,113.18,106.24,111.56,https://www.numbeo.com/cost-of-living/in/Lausanne,CHE
6,Geneva,None,Switzerland,"Geneva, Switzerland",103.97,70.36,88.07,102.07,110.93,111.93,https://www.numbeo.com/cost-of-living/in/Geneva,CHE
7,Honolulu,US-HI,United States,"Honolulu, HI, United States",101.81,63.28,83.58,112.58,86.98,93.82,https://www.numbeo.com/cost-of-living/in/Honolulu,USA
8,Beirut,None,Lebanon,"Beirut, Lebanon",100.94,24.66,64.85,118.67,71.87,14.26,https://www.numbeo.com/cost-of-living/in/Beirut,LBN
9,Santa Barbara,US-CA,United States,"Santa Barbara, CA, United States",100.05,90.86,95.70,107.15,95.94,69.60,https://www.numbeo.com/cost-of-living/in/Santa...,USA


Эксперименты с приведение штатов к стандарту. Сработало.

In [93]:
#df34 = df.copy()
#for i, state in df34['state'].items():
#    if state != None:
#        df34.loc[i, 'state'] = f'US-{state}'

### Scraping US-states codes from wiki and transform

In [94]:
url_states = 'https://en.wikipedia.org/wiki/ISO_3166-2:US'

In [95]:
df_states = pd.read_html(url_states)[0]

In [96]:
df_states.head()

,Code,Subdivision name (en),Subdivision category
0,US-AL,Alabama,State
1,US-AK,Alaska,State
2,US-AZ,Arizona,State
3,US-AR,Arkansas,State
4,US-CA,California,State


In [97]:
df_states.columns = ['state_code', 'state_name', 'category']
df_states.set_index('state_code', inplace=True)

In [98]:
df_states.head()

,state_name,category
state_code,,
US-AL,Alabama,State
US-AK,Alaska,State
US-AZ,Arizona,State
US-AR,Arkansas,State
US-CA,California,State


# Карта
В индексе Numbeo 505 городов, но не понятно, какие страны, много их или мало. Сколько городов в этих странах и в каких пропорциях. Суммарные цифры мне мало что скажут, гораздо понятнее будет отобразить их на карте.

In [99]:
map_df = df.groupby('country_code')['city'].nunique().reset_index().sort_values('city', ascending=False)
map_df

,country_code,city
119,USA,74
40,GBR,29
18,CAN,28
28,DEU,22
56,ITA,18
...,...,...
59,JOR,1
58,JEY,1
57,JAM,1
54,ISL,1


In [101]:
m = folium.Map(location=[37.87820990704326, 6.555063556986549], zoom_start=1.5)
folium.Choropleth(
    geo_data='data/world.geojson',
    name="choropleth",
    data=map_df,
    columns=['country_code', 'city'],
    key_on="feature.properties.ISO_A3", 
    fill_color="YlGn",
    nan_fill_color='pink',
    fill_opacity=0.8,
    bins=7,
    reset=True,
    highlight=True,
    legend_name='Count of cities'
).add_to(m)

На github интерактивная карта не отображается, поэтому вместо вывода карты, вставил картинку.

Теперь видно какие страны есть в индексе, а каких нет, и сколько городов в каждой стране. Меня всё устраивает, вот только мне не нужно в Индии много городов. Удалю лишние (оставлю 2).

![title](map.png)